In [8]:
import pandas as pd

In [9]:
hof = pd.read_csv('hof.csv')
# only keep players who were selected for the hall of fame
hof = hof[hof['In_Hall_of_fame'] == 1]
hof.head()

,PLAYER_NAME,position,All_star_selections,In_Hall_of_fame,height,weight,born
244,Kareem Abdul-Jabbar,C,19,1,218,102,1947
245,Shaquille O'Neal,C,15,1,216,147,1972
246,Wilt Chamberlain,C,13,1,216,124,1936
247,Moses Malone,C,12,1,208,97,1955
248,Bill Russell,C,12,1,208,97,1934


In [10]:
# get all csvs from player_stats folder
import os
player_stats_path = os.path.join('player_stats', 'player_stats.csv')
salaries_path = os.path.join('player_stats', 'salaries.csv')

# read all csvs into dataframes
player_stats = pd.read_csv(player_stats_path)
salaries = pd.read_csv(salaries_path)

In [11]:
#player_box_score.drop(['Unnamed: 0', 'VIDEO_AVAILABLE'], axis=1, inplace=True)
player_stats.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
salaries.drop(['Unnamed: 0'], axis=1, inplace=True)

In [12]:
# merge into player_stats with player_stats and salaries df by 'PLAYER_NAME' and 'Season', if no salary exists fill with null
player_stats = player_stats.merge(salaries, how='left', on=['PLAYER_NAME', 'Season'])

In [13]:
player_stats.columns

Index(['Season', 'PLAYER_NAME', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'salary', 'inflationAdjSalary'],
      dtype='object')

In [14]:
player_stats.tail()

,Season,PLAYER_NAME,Pos,Age,Tm,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,salary,inflationAdjSalary
28765,2022,Thaddeus Young,PF,33.0,TOR,26.0,0.0,475.0,67.0,144.0,...,75.0,115.0,45.0,31.0,11.0,22.0,43.0,164.0,NaN,NaN
28766,2022,Trae Young,PG,23.0,ATL,76.0,76.0,2652.0,711.0,1544.0,...,234.0,284.0,737.0,72.0,7.0,303.0,128.0,2155.0,NaN,NaN
28767,2022,Omer Yurtseven,C,23.0,MIA,56.0,12.0,706.0,130.0,247.0,...,209.0,294.0,49.0,17.0,20.0,41.0,84.0,299.0,NaN,NaN
28768,2022,Cody Zeller,C,29.0,POR,27.0,0.0,355.0,51.0,90.0,...,75.0,125.0,22.0,8.0,6.0,19.0,56.0,140.0,NaN,NaN
28769,2022,Ivica Zubac,C,24.0,LAC,76.0,76.0,1852.0,310.0,495.0,...,427.0,644.0,120.0,36.0,77.0,114.0,203.0,785.0,NaN,NaN


In [15]:
# remove $ and convert salary and inflationAdjSalary to float
player_stats['salary'] = player_stats['salary'].str.replace('$', '')
player_stats['salary'] = player_stats['salary'].str.replace(',', '')
player_stats['inflationAdjSalary'] = player_stats['inflationAdjSalary'].str.replace('$', '')
player_stats['inflationAdjSalary'] = player_stats['inflationAdjSalary'].str.replace(',', '')
player_stats['salary'] = player_stats['salary'].astype(float)
player_stats['inflationAdjSalary'] = player_stats['inflationAdjSalary'].astype(float)

In [16]:
# create summary_df which contains the sum of 'G', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS' for each player
# also add the average of 'FG%', '3P%', '2P%', 'eFG%', 'FT%', 'salary', 'inflationAdjSalary' for each player
# also add the number of seasons played for each player
to_sum = ['G', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
player_dict = {}
# create a list of 18 empty lists
for row in player_stats.iterrows():
    row = row[1]
    cur = player_dict.get(row['PLAYER_NAME'], [[] for i in range(len(to_sum))])
    for index, col in enumerate(to_sum):
        cur[index].append(row[col])
    player_dict[row['PLAYER_NAME']] = cur

player_dict_new = {}
for player, values in player_dict.items():
    player_dict_new[player] = [sum(values[i]) for i in range(len(values))]

player_dict = player_dict_new
# remember to add max and min salaries at end of loop, and compute percentages
player_dict['Kobe Bryant']
cols = ['PLAYER_NAME'] + to_sum
player_list = [[key] + value for key, value in player_dict.items()]

summary_df = pd.DataFrame(player_list, columns=cols)
summary_df.head()

,PLAYER_NAME,G,FG,FGA,3P,3PA,2P,2PA,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Curly Armstrong,101.0,216.0,748.0,NaN,NaN,216.0,748.0,228.0,331.0,NaN,NaN,NaN,253.0,NaN,NaN,NaN,314.0,660.0
1,Cliff Barker,149.0,201.0,637.0,NaN,NaN,201.0,637.0,155.0,234.0,NaN,NaN,NaN,294.0,NaN,NaN,NaN,253.0,557.0
2,Leo Barnhorst,416.0,1555.0,4277.0,NaN,NaN,1555.0,4277.0,583.0,870.0,NaN,NaN,NaN,1342.0,NaN,NaN,NaN,1236.0,3693.0
3,Ed Bartels,47.0,68.0,269.0,NaN,NaN,68.0,269.0,62.0,114.0,NaN,NaN,NaN,52.0,NaN,NaN,NaN,112.0,198.0
4,Ralph Beard,126.0,749.0,2046.0,NaN,NaN,749.0,2046.0,508.0,660.0,NaN,NaN,NaN,551.0,NaN,NaN,NaN,228.0,2006.0


In [17]:
# add 'FG%', '3P%', '2P%', 'eFG%', 'FT%', 'ORBPG', 'DRBPG', 'TRBPG', 'ASTPG', 'STLPG', 'BLKPG', 'TOVPG', 'PFPG', 'PTSPG',
summary_df['FG%'] = summary_df['FG'] / summary_df['FGA']
summary_df['3P%'] = summary_df['3P'] / summary_df['3PA']
summary_df['2P%'] = summary_df['2P'] / summary_df['2PA']
summary_df['eFG%'] = (summary_df['FG'] + 0.5 * summary_df['3P']) / summary_df['FGA']
summary_df['FT%'] = summary_df['FT'] / summary_df['FTA']
summary_df['ORBPG'] = summary_df['ORB'] / summary_df['G']
summary_df['DRBPG'] = summary_df['DRB'] / summary_df['G']
summary_df['TRBPG'] = summary_df['TRB'] / summary_df['G']
summary_df['ASTPG'] = summary_df['AST'] / summary_df['G']
summary_df['STLPG'] = summary_df['STL'] / summary_df['G']
summary_df['BLKPG'] = summary_df['BLK'] / summary_df['G']
summary_df['TOVPG'] = summary_df['TOV'] / summary_df['G']
summary_df['PFPG'] = summary_df['PF'] / summary_df['G']
summary_df['PTSPG'] = summary_df['PTS'] / summary_df['G']
summary_df.head()

,PLAYER_NAME,G,FG,FGA,3P,3PA,2P,2PA,FT,FTA,...,FT%,ORBPG,DRBPG,TRBPG,ASTPG,STLPG,BLKPG,TOVPG,PFPG,PTSPG
0,Curly Armstrong,101.0,216.0,748.0,NaN,NaN,216.0,748.0,228.0,331.0,...,0.688822,NaN,NaN,NaN,2.504950,NaN,NaN,NaN,3.108911,6.534653
1,Cliff Barker,149.0,201.0,637.0,NaN,NaN,201.0,637.0,155.0,234.0,...,0.662393,NaN,NaN,NaN,1.973154,NaN,NaN,NaN,1.697987,3.738255
2,Leo Barnhorst,416.0,1555.0,4277.0,NaN,NaN,1555.0,4277.0,583.0,870.0,...,0.670115,NaN,NaN,NaN,3.225962,NaN,NaN,NaN,2.971154,8.877404
3,Ed Bartels,47.0,68.0,269.0,NaN,NaN,68.0,269.0,62.0,114.0,...,0.543860,NaN,NaN,NaN,1.106383,NaN,NaN,NaN,2.382979,4.212766
4,Ralph Beard,126.0,749.0,2046.0,NaN,NaN,749.0,2046.0,508.0,660.0,...,0.769697,NaN,NaN,NaN,4.373016,NaN,NaN,NaN,1.809524,15.920635


In [18]:
player_names = summary_df['PLAYER_NAME']
# get the number of seasons played for each player
seasons_played = []
for player in player_names:
    seasons_played.append(len(player_stats[player_stats['PLAYER_NAME'] == player]))

summary_df['seasons_played'] = seasons_played

In [19]:
# merge hof into summary_df on PLAYER_NAME
summary_df = summary_df.merge(hof, how='left', on='PLAYER_NAME')
summary_df.fillna(0, inplace=True)
drop = ['height', 'weight', 'born', 'position']
summary_df.drop(drop, axis=1, inplace=True)
summary_df.head()


,PLAYER_NAME,G,FG,FGA,3P,3PA,2P,2PA,FT,FTA,...,TRBPG,ASTPG,STLPG,BLKPG,TOVPG,PFPG,PTSPG,seasons_played,All_star_selections,In_Hall_of_fame
0,Curly Armstrong,101.0,216.0,748.0,0.0,0.0,216.0,748.0,228.0,331.0,...,0.0,2.504950,0.0,0.0,0.0,3.108911,6.534653,2,0.0,0.0
1,Cliff Barker,149.0,201.0,637.0,0.0,0.0,201.0,637.0,155.0,234.0,...,0.0,1.973154,0.0,0.0,0.0,1.697987,3.738255,3,0.0,0.0
2,Leo Barnhorst,416.0,1555.0,4277.0,0.0,0.0,1555.0,4277.0,583.0,870.0,...,0.0,3.225962,0.0,0.0,0.0,2.971154,8.877404,7,0.0,0.0
3,Ed Bartels,47.0,68.0,269.0,0.0,0.0,68.0,269.0,62.0,114.0,...,0.0,1.106383,0.0,0.0,0.0,2.382979,4.212766,4,0.0,0.0
4,Ralph Beard,126.0,749.0,2046.0,0.0,0.0,749.0,2046.0,508.0,660.0,...,0.0,4.373016,0.0,0.0,0.0,1.809524,15.920635,2,0.0,0.0


In [20]:
def show_player_stats(player, summary_df):
    return summary_df[summary_df['PLAYER_NAME'] == player]

In [21]:
most_recent_season = player_stats['Season'].max()
# create a new df with only the most recent season
player_stats_recent = player_stats[player_stats['Season'] == most_recent_season]
# get list of names of players who played in the most recent season
recent_players = player_stats_recent['PLAYER_NAME'].unique()
# create a new column in summary df called 'retired' if player is not in recent_players then they are retired
summary_df['retired'] = summary_df['PLAYER_NAME'].apply(lambda x: 1 if x not in recent_players else 0)
summary_df.head()

,PLAYER_NAME,G,FG,FGA,3P,3PA,2P,2PA,FT,FTA,...,ASTPG,STLPG,BLKPG,TOVPG,PFPG,PTSPG,seasons_played,All_star_selections,In_Hall_of_fame,retired
0,Curly Armstrong,101.0,216.0,748.0,0.0,0.0,216.0,748.0,228.0,331.0,...,2.504950,0.0,0.0,0.0,3.108911,6.534653,2,0.0,0.0,1
1,Cliff Barker,149.0,201.0,637.0,0.0,0.0,201.0,637.0,155.0,234.0,...,1.973154,0.0,0.0,0.0,1.697987,3.738255,3,0.0,0.0,1
2,Leo Barnhorst,416.0,1555.0,4277.0,0.0,0.0,1555.0,4277.0,583.0,870.0,...,3.225962,0.0,0.0,0.0,2.971154,8.877404,7,0.0,0.0,1
3,Ed Bartels,47.0,68.0,269.0,0.0,0.0,68.0,269.0,62.0,114.0,...,1.106383,0.0,0.0,0.0,2.382979,4.212766,4,0.0,0.0,1
4,Ralph Beard,126.0,749.0,2046.0,0.0,0.0,749.0,2046.0,508.0,660.0,...,4.373016,0.0,0.0,0.0,1.809524,15.920635,2,0.0,0.0,1


In [22]:
show_player_stats('Kobe Bryant', summary_df)

,PLAYER_NAME,G,FG,FGA,3P,3PA,2P,2PA,FT,FTA,...,ASTPG,STLPG,BLKPG,TOVPG,PFPG,PTSPG,seasons_played,All_star_selections,In_Hall_of_fame,retired
2457,Kobe Bryant,1346.0,11719.0,26200.0,1827.0,5546.0,9892.0,20654.0,8378.0,10011.0,...,4.684993,1.444279,0.475483,2.979198,2.491085,24.994799,20,18.0,1.0,1


In [23]:
show_player_stats('LeBron James', summary_df)

,PLAYER_NAME,G,FG,FGA,3P,3PA,2P,2PA,FT,FTA,...,ASTPG,STLPG,BLKPG,TOVPG,PFPG,PTSPG,seasons_played,All_star_selections,In_Hall_of_fame,retired
2946,LeBron James,1411.0,13965.0,27648.0,2244.0,6471.0,11721.0,21177.0,8014.0,10928.0,...,7.367116,1.547838,0.755493,3.512403,1.832743,27.064493,20,0.0,0.0,0


In [24]:
# convert In_Hall_of_fame to int
summary_df['In_Hall_of_fame'] = summary_df['In_Hall_of_fame'].astype(int)

# Logistic Regression
Now we train the model on the data we have gathered

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

def create_model_for_player(player_name, summary_df):
    # drop row with player_name
    summary_df = summary_df[summary_df['PLAYER_NAME'] != player_name]
    X = summary_df.drop(['In_Hall_of_fame','PLAYER_NAME'], axis=1)
    # get index of each player name into list
    y = summary_df['In_Hall_of_fame']
    logreg = LogisticRegression()
    logreg.fit(X, y)
    return logreg

def predict_player(player, summary_df):
    player_stats = show_player_stats(player, summary_df)
    # set player retired
    player_stats['retired'] = 1
    player_stats.drop(['In_Hall_of_fame'], axis=1, inplace=True)
    player_stats.drop(['PLAYER_NAME'], axis=1, inplace=True)
    player_stats = player_stats.values
    player_stats = player_stats.reshape(1, -1)
    model = create_model_for_player(player, summary_df)
    return model.predict(player_stats)


In [33]:
active_players_df = summary_df[summary_df['retired'] == 0].copy()
active_players_df.head()

,PLAYER_NAME,G,FG,FGA,3P,3PA,2P,2PA,FT,FTA,...,ASTPG,STLPG,BLKPG,TOVPG,PFPG,PTSPG,seasons_played,All_star_selections,In_Hall_of_fame,retired
266,George King,416.0,1517.0,3976.0,0.0,0.0,1517.0,3972.0,1186.0,1849.0,...,4.706731,0.000000,0.000000,0.000000,2.519231,10.144231,8,0.0,0,0
2581,Brandon Williams,42.0,122.0,339.0,34.0,116.0,88.0,223.0,72.0,105.0,...,2.309524,0.761905,0.309524,1.666667,1.761905,8.333333,4,0.0,0,0
2818,Joe Johnson,1490.0,8597.0,19523.0,2176.0,5919.0,6421.0,13604.0,2982.0,3708.0,...,3.745638,0.811409,0.195302,1.832215,1.730872,15.001342,24,0.0,0,0
2908,Carmelo Anthony,1406.0,11130.0,24923.0,1959.0,5449.0,9171.0,19474.0,6964.0,8523.0,...,2.665007,0.950925,0.517781,2.360597,2.781650,22.178521,22,0.0,0,0
2941,Udonis Haslem,873.0,2661.0,5435.0,6.0,62.0,2655.0,5373.0,1235.0,1634.0,...,0.839633,0.484536,0.286369,0.926690,2.357388,7.517755,20,0.0,0,0


In [70]:
# Read the 'finals_mvp.csv' and 'nba_mvps.csv' files
finals_mvps = pd.read_csv("finals_mvp.csv")
regular_mvps = pd.read_csv("nba_mvps.csv")

# Initialize the new columns in summary_df with zeros
summary_df['regular_mvp_count'] = 0
summary_df['finals_mvp_count'] = 0

# Increment the MVP counts based on the corresponding names in finals_mvps and regular_mvps
for index, row in finals_mvps.iterrows():
    player_name = row['PLAYER_NAME']
    summary_df.loc[summary_df['PLAYER_NAME'] == player_name, 'finals_mvp_count'] += 1

for index, row in regular_mvps.iterrows():
    player_name = row['PLAYER_NAME']
    summary_df.loc[summary_df['PLAYER_NAME'] == player_name, 'regular_mvp_count'] += 1

# Filter the rows where either of the counts is greater than 0
filtered_summary_df = summary_df[
    (summary_df['regular_mvp_count'] > 0) | (summary_df['finals_mvp_count'] > 0)
]

# Print the filtered summary_df
filtered_summary_df.head()


,PLAYER_NAME,G,FG,FGA,3P,3PA,2P,2PA,FT,FTA,...,BLKPG,TOVPG,PFPG,PTSPG,seasons_played,All_star_selections,In_Hall_of_fame,retired,regular_mvp_count,finals_mvp_count
232,Bob Cousy,924.0,6168.0,16468.0,0.0,0.0,6168.0,16468.0,4624.0,5756.0,...,0.0,0.0,2.426407,18.354978,14,13.0,1,1,1,0
378,Bob Pettit,792.0,7349.0,16872.0,0.0,0.0,7349.0,16872.0,6182.0,8119.0,...,0.0,0.0,3.193182,26.363636,11,11.0,1,1,2,0
437,Bill Russell,963.0,5687.0,12930.0,0.0,0.0,5687.0,12930.0,3148.0,5614.0,...,0.0,0.0,2.691589,15.079958,13,12.0,1,1,5,0
493,Wilt Chamberlain,1118.0,13744.0,25580.0,0.0,0.0,13744.0,25580.0,6465.0,12742.0,...,0.0,0.0,1.986583,30.369410,16,13.0,1,1,4,1
524,Oscar Robertson,1040.0,9508.0,19620.0,0.0,0.0,9508.0,19620.0,7694.0,9185.0,...,0.0,0.0,2.818269,25.682692,14,12.0,1,1,1,0


In [71]:
import pandas as pd

# Create an empty DataFrame to store the predictions for current players
current_players_predictions = pd.DataFrame(columns=['PLAYER_NAME', 'Prediction'])

# Iterate over the PLAYER_NAME column in active_players_df
for player in active_players_df["PLAYER_NAME"]:
    # Call predict_player function for each player and store the result in predictions
    prediction = predict_player(player, summary_df)
    row = {'PLAYER_NAME': player, 'Prediction': prediction[0]}
    current_players_predictions = pd.concat([current_players_predictions, pd.DataFrame([row])])

# Reset the row index
current_players_predictions.reset_index(drop=True, inplace=True)

# Print the predictions for current players
current_players_predictions


C:\Users\moham\AppData\Local\Temp\ipykernel_3884\2932204295.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_stats['retired'] = 1
C:\Users\moham\AppData\Local\Temp\ipykernel_3884\2932204295.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_stats.drop(['In_Hall_of_fame'], axis=1, inplace=True)
C:\Users\moham\AppData\Local\Temp\ipykernel_3884\2932204295.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

,PLAYER_NAME,Prediction
0,George King,0
1,Brandon Williams,0
2,Joe Johnson,0
3,Carmelo Anthony,0
4,Udonis Haslem,0
...,...,...
600,Ziaire Williams,0
601,McKinley Wright IV,0
602,Moses Wright,0
603,Gabe York,0


In [72]:
# Print only the players with a positive prediction

positive_predictions = current_players_predictions[current_players_predictions['Prediction'] == 1]
positive_predictions

,PLAYER_NAME,Prediction
5,LeBron James,1
10,Chris Paul,1
19,Kevin Durant,1
35,Russell Westbrook,1
37,Stephen Curry,1
42,James Harden,1
51,DeMarcus Cousins,1
84,Anthony Davis,1
97,Giannis Antetokounmpo,1
157,Nikola Jokić,1


In [73]:
# set X and y
from sklearn.model_selection import train_test_split
X = summary_df.drop(['In_Hall_of_fame', 'PLAYER_NAME'], axis=1)


y = summary_df['In_Hall_of_fame']
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# create logistic regression model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
accuracy_score(y_test, y_pred)

C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.987750556792873